In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re

import pandas as pd

import kraft

In [ ]:
SETTING = kraft.json.read("setting.json")

In [ ]:
directory_path = "{}/protein/".format(SETTING["directory_path"])

kraft.path.path(directory_path)

In [ ]:
for url in (
    "https://gygi.med.harvard.edu/sites/gygi.med.harvard.edu/files/documents/protein_quant_current_normalized.csv.gz",
):

    print(kraft.internet.download(url, directory_path))

In [ ]:
gene_x_cell_line = pd.read_csv(
    "{}/protein_quant_current_normalized.csv.gz".format(directory_path), index_col=1
).drop(columns=["Protein_Id", "Description", "Group_ID", "Uniprot", "Uniprot_Acc"])

print(gene_x_cell_line.shape)

gene_x_cell_line = gene_x_cell_line.loc[~gene_x_cell_line.index.isna(), :]

print(gene_x_cell_line.shape)

gene_x_cell_line = gene_x_cell_line.groupby(level=0).median()

print(gene_x_cell_line.shape)

gene_x_cell_line = gene_x_cell_line.loc[
    :,
    (
        not bool(re.match("^TenPx", label))
        for label in gene_x_cell_line.columns.to_numpy()
    ),
]

gene_x_cell_line.columns = kraft.cell_line.rename(
    (
        re.sub("_TenPx[0-9]{2}$", "", label)
        for label in gene_x_cell_line.columns.to_numpy()
    )
)

print(gene_x_cell_line.shape)

gene_x_cell_line = gene_x_cell_line.groupby(axis=1, level=0).median()

print(gene_x_cell_line.shape)

gene_x_cell_line.index.name = "Gene"

gene_x_cell_line.columns.name = "Cell Line"

kraft.dataframe.error_axes(gene_x_cell_line)

gene_x_cell_line.to_csv("{}/gene_x_cell_line.tsv".format(directory_path), sep="\t")

gene_x_cell_line